# 텍스트 분류 5차 시도 ('O'인 행만 추려서 '구분'칼럼성분 분류하기)

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from konlpy.tag import Okt, Kkma, Mecab, Hannanum
Okt = Okt()
kkma = Kkma()
hannanum = Hannanum()

In [3]:
df = pd.read_csv('final_text_data2.csv', sep='\t', encoding='UTF-8')

In [4]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인', '검토결과'],
      dtype='object')

In [5]:
df.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   
3               W  2018     20180928         ZZ7         B11  조립 문제_미장착/오장착   
4               W  2016     20160526         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   
3         KV11          YP     A9          TT        83120   
4         KV11          YP     A9          TT        83860   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  \
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN   
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN   
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN   
3              [C] 현상및조치   차량출고시 웨즈스트립고무 주름발생 들뜸교환조치     X  NaN  NaN  NaN   
4                     [C] 몰딩 내부 고무 미조립 상태에서 몰딩 조립 출고     X  NaN  NaN  NaN   

   검토결과  
0  X---  
1  X---  
2  X---  
3  X---  
4  X---

In [6]:
df.shape

(10629, 17)

# 분류현황

In [7]:
category_count = df['대상 선정'].value_counts()
category_count

X    5519
O    3013
Name: 대상 선정, dtype: int64

In [8]:
category_count = df['책임구'].value_counts()
category_count

구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [9]:
category_count = df['원인'].value_counts()
category_count

부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

# null인 값 확인 및 제거

In [10]:
df_bak = df.copy()

In [11]:
# 제거
# df = df[df['대상 선정'].notnull()].copy()
# 다른값으로 채우기
df = df.fillna('N')

In [12]:
df.shape

(10629, 17)

In [13]:
category_count = df['대상 선정'].value_counts()
category_count

X    5519
O    3013
N    2097
Name: 대상 선정, dtype: int64

In [14]:
category_count = df['책임구'].value_counts()
category_count

N       7617
구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [15]:
category_count = df['원인'].value_counts()
category_count

N        7617
부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

In [16]:
df2 = df.loc[df['대상 선정']=='O',:] # 대상 선정값이 O 인 것만 가져오기 (X제외)
                                     
# 분류가 발생돼야 하는 값은 '대상 선정'이 O 인 값이다.
# O 인 값은 3013행이다. 나머지값(X:5519,N:2097)에서 'clm_prtl_sbs'값도 중요하지만결국 그 RO내용은 정작 중요한 값은 아닐 것이다.
# 굳이 중요한 RO문장도 아닌데 포함시킬 필요가 있을까? 
# 그래서 구분,책임구,책임구,원인에 대한 분류는 대상선정값이 O인 값을 중심으로 분류해본다.

In [17]:
df2_bak = df2.copy()

In [18]:
df2 = df2.reindex()

In [19]:
df2.shape

(3013, 17)

In [20]:
df2.head(50)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd  \
12                 W  2017     20171010         ZZ7         B11   
14                 W  2018     20180725         ZZ7         B11   
15                 W  2015     20151114         ZZ3         B11   
19                 W  2017     20170522         ZZ5         B11   
24                 W  2016     20160425         ZZ5         B11   
25                 W  2015     20151201         ZZ5         B11   
32                 W  2018     20180302         ZZ7         B11   
38                 W  2015     20150514         ZZ7         B11   
40                 W  2016     20160810         ZZ4         B11   
58                 W  2015     20151003         ZZ5         B11   
63                 W  2017     20170317         ZZ5         B11   
66                 W  2016     20161009         ZZ5         B11   
77                 W  2018     20181123         ZZ3         B11   
92                 W  2016     20160412         ZZ5         B11   
106                W  2016     20160718         ZZ7         B11   
108                W  2015     20150730         ZZ5         B11   
127                W  2017     20180110         ZZ7         B11   
141                W  2015     20151130         ZZ7         B11   
147                W  2017     20170714         ZZ3         B11   
156                W  2017     20171016         ZZ3         B11   
164                W  2018     20180430         ZZ3         B11   
170                W  2015     20150203         ZZ5         B11   
171                W  2015     20151022         ZZ3         B11   
173                W  2017     20170224         ZZ3         B11   
177                W  2017     20170908         ZZ5         B11   
179                W  2015     20150910         ZZ5         B11   
185                W  2016     20161227         ZZ5         B11   
189                W  2016     20160110         ZZ7         B11   
192                W  2015     20150812         ZZ7         B11   
209                W  2017     20170911         ZZ7         B11   
210                W  2016     20160325         ZZ7         B11   
220                W  2018     20181030         ZZ7         B11   
221                W  2016     20160928         ZZ7         B11   
276                W  2018     20180525         ZZ7         B12   
304                W  2015     20150922         ZZ5         B12   
323                W  2015     20151130         ZZ5         B12   
346                W  2018     20180322         ZZ5         B12   
423                W  2016     20160804         ZZ7         B12   
463                W  2015     20150825         ZZ5         B12   
477                W  2016     20160324         ZZ3         B12   
576                W  2015     20151229         ZZ7         B12   
581                W  2017     20170704         ZZ7         B12   
591                W  2018     20181106         ZZ7         B12   
604                W  2017     20170608         ZZ3         B12   
673                W  2015     20151117         ZZ7         B12   
851                W  2018     20180329         ZZ5         B12   
1013               W  2018     20180919         ZZ7         B13   
1014               W  2018     20181129         ZZ5         B13   
1015               W  2015     20160107         ZZ4         B13   
1029               W  2017     20170120         ZZ7         B13   

        ceq_psit_nm trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
12    조립 문제_미장착/오장착         KV11          CK     J5          AC        91500   
14    조립 문제_미장착/오장착         KV11          YP     A9          EE        28291   
15    조립 문제_미장착/오장착         KV11          YP     A9          TT        81750   
19    조립 문제_미장착/오장착         KV11          YP     A9          AC        95910   
24    조립 문제_미장착/오장착         KV11          YP     A9          AC        91501   
25    조립 문제_미장착/오장착         KV11          KH     DH          TT        87610   
32    조립 문제_미장착/오장착        

# 전처리

In [21]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df2['dif_pred'] = df2['구분'].astype(str).copy()
# df2['charge_pred'] = df2['책임구'].astype(str).copy()
# df2['cause_pred'] = df2['원인'].astype(str).copy()

In [22]:
def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text) # 특수문자 제거
    #text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)   # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)   # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)   # 한글, 영문만 남기고 모두 제거하도록 합니다.
    return text

In [23]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

In [24]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(preprocessing)

Wall time: 31.2 ms


In [25]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 109 ms


In [26]:
df2['clm_prtl_sbc'].head(10)

12          가  인도후 오디오 작동시 주파수 및 노이즈 발생으로 불만 제기   및 ...
14          주행중 엔진경고등 점등   및  스캔점검결과 P       DPF효율저하...
15      테일게이트트림에서 뜨르르르 이상소음발생으로 테일게이트트림 탈착하여 확인시 클립이...
19                     조수석사이드에어백 경고등 점등됨 컨넥터 체결불량으로 재체결
24                        USB 작동 안됨원인 USB 컨넥터 조립 않됨 재조립
25                       어라운드뷰 장착차량인데 운전석 미러에는 카메라 미장착됨
32        리어LH윈도우작동불량  점검시 뒤운전석쪽만 작동불량 퓨즈확인시정상   트림탈...
38      운전석 뒤도어 유리창 작동 안되어  메인파워스위치 배선인가 점검후뒷도어트림스위치...
40       상기차량 주행중 운전석 시트가 들석이는 현상발생입고 점검시 운전석 앞 우측 시...
58           와이퍼 스몰등 후방카메라 오작동 불량   및   리어 루프 배선 R  ...
Name: clm_prtl_sbc, dtype: object

# 학습데이터와 테스트데이터 만들기

In [27]:
df = df.reindex() #index 재설정. reindex:새로운 색인에 맞도록 객체를 새로 생성하는 기능.

In [28]:
df2.shape

(3013, 18)

In [29]:
split_count = int(df2.shape[0] * 0.8)
split_count

2410

In [30]:
df_train = df2[:split_count].copy()
df_train.shape

(2410, 18)

In [31]:
df_test = df2[split_count:].copy()
df_test.shape

(603, 18)

# 단어 벡터화

In [32]:
from sklearn.feature_extraction.text import CountVectorizer

stops = ['[C]']
vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor = None, # 전처리 도구
                             stop_words = stops, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
                             min_df = 1, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3), # BOW의 단위를 1~3개로 지정합니다.
                             max_features = 12900 # 만들 피처의 수, 단어의 수가 된다.
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=12900, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=['[C]'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [33]:
#%%time 
train_feature_vector = vectorizer.fit_transform(df_train['clm_prtl_sbc'])
train_feature_vector.shape

(2410, 12900)

In [34]:
%%time 
test_feature_vector = vectorizer.fit_transform(df_test['clm_prtl_sbc'])
test_feature_vector.shape

Wall time: 93.6 ms


In [35]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

12900


['a필라',
 'a필러',
 'a필러 트립',
 'a필러 트립 교환',
 'a필러 트립 유격',
 'a필러측',
 'b필러',
 'h이상',
 'km',
 'km h이상']

In [36]:
dist = np.sum(train_feature_vector, axis=0)

pd.DataFrame(dist, columns=vocab)

a필라  a필러  a필러 트립  a필러 트립 교환  a필러 트립 유격  a필러측  b필러  h이상  km  km h이상 ...  \
0   28    3       2          3          7     2    4    2   2       2 ...   

   힌지 재조임 전도어  힌지의  힌지의 뒤쪽  힌지의 뒤쪽 스톱퍼이  힌지의 스톱퍼  힌지의 스톱퍼 볼트를  힘가하면  \
0           2    2       2            2        2            4     7   

   힘가하면 소리않남  힘가하면 소리않남 오버헤드  힘을  
0          2               2   2  

[1 rows x 12900 columns]

# tf-idf 가중치 적용

In [51]:
# tf-idf(Term Frequency-Inverse Document Frequency). 가중치를 구하는 알고리즘.

In [37]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [38]:
#%%time 
train_feature_tfidf = transformer.fit_transform(train_feature_vector)
train_feature_tfidf.shape

(2410, 12900)

In [39]:
%%time 
test_feature_tfidf = transformer.fit_transform(test_feature_vector)
test_feature_tfidf.shape

Wall time: 0 ns


# 머신러닝 : 학습

In [40]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [41]:
# 학습에 사용할 y_label 을 넣어줍니다.
# 어떤 분야의 청원인지 예측할 것이기 때문에 category를 넣어줍니다.
# y_label2 = df_train['caus_pred']
y_label = df_train['dif_pred']
# y_label3 = df_train['charge_pred']
%time forest = forest.fit(train_feature_tfidf, y_label)
# %time forest2 = forest.fit(train_feature_tfidf, y_label2)
# %time forest3 = forest.fit(train_feature_tfidf, y_label3)

Wall time: 406 ms


# 모델평가

In [42]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, train_feature_vector, y_label, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 21.4 s


array([0.93360996, 0.92946058, 0.89626556, 0.89004149, 0.91493776])

In [43]:
round(np.mean(score)*100,2)

91.29

# 예측

In [44]:
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(test_feature_vector)
y_pred[:3]

array(['기능(커넥터)', '기능(커넥터)', '실내부품이음'], dtype=object)

In [45]:
y_pred.shape

(603,)

In [46]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 줍니다.
output = pd.DataFrame(data={'category_pred':y_pred})
output.head(10)

category_pred
0       기능(커넥터)
1       기능(커넥터)
2        실내부품이음
3       기능(커넥터)
4       기능(커넥터)
5       기능(커넥터)
6       기능(커넥터)
7       기능(커넥터)
8       기능(커넥터)
9       기능(커넥터)

In [47]:
df_test['category_pred'] = y_pred

# 결과검증

In [48]:
df_test['pred_diff'] = 0
df_test['pred_diff'] = (df_test['구분'] == df_test['category_pred'] ) == 1
df_test['pred_diff'] = df_test['pred_diff'].astype(int)
df_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd ceq_psit_nm  \
8078               W  2017     20170607         ZZ5         Q55   소음/이음_주행중   
8083               W  2017     20170429         ZZ5         Q55   소음/이음_주행중   
8087               W  2018     20180731         ZZ3         Q55   소음/이음_주행중   
8088               W  2017     20170608         ZZ5         Q55   소음/이음_주행중   
8091               W  2016     20160328         ZZ5         Q55   소음/이음_주행중   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
8078         KV11          YP     A9          TT        87321   
8083         KV11          YP     A9          CC        28710   
8087         KV11          YP     A9          TT        82540   
8088         KV11          YP     A9          TT        88400   
8091         KV11          YP     A9          CC        97113   

                                           clm_prtl_sbc 대상 선정      구분   책임구  \
8078         주행중 R 테일게이트 소음발생으로 웨더스트립고무 교환및 스트라키커조정 완료함     O  실내부품이음  생산공장   
8083       주행중 머플러음 소리 큼   점검  내 외관 점검 및 테스트시 중부하음 두...     O    하체이음    구매   
8087                         프론트 도어 조수석 런 들뜸 현상단품자체불량교환     O  실내부품이음  생산공장   
8088         가  주행시 소음 발생으로 입고   점검및   가  시운전시 시트 백에...     O  실내부품이음    구매   
8091                   히터작동중 간헐적 블로어모터이음발생 확인되어 블로어모터교환     O  실내부품이음    구매   

        원인                검토결과 dif_pred category_pred  pred_diff  
8078  조립불량  O-실내부품이음-생산공장-조립불량   실내부품이음       기능(커넥터)          0  
8083    부품        O-하체이음-구매-부품     하체이음       기능(커넥터)          0  
8087  조립불량  O-실내부품이음-생산공장-조립불량   실내부품이음        실내부품이음          1  
8088    부품      O-실내부품이음-구매-부품   실내부품이음       기능(커넥터)          0  
8091    부품      O-실내부품이음-구매-부품   실내부품이음       기능(커넥터)          0

In [49]:
diff_sum = df_test['pred_diff'].sum()
diff_sum

57

In [50]:
y_pred_len = len(y_pred)
y_pred_len

603

# 정확도

In [51]:
Acy = round(diff_sum/y_pred_len*100,1)
Acy

9.5